# Final project
## Sangdae Kim
#### Supporting notebook for Ads for tourists vs locals inside Yellow cabs

##### This notebook do lots of drawing & calculations, run by each cell please.
##### This project uses pyproj and gmaps, it may needs to be installed(pip).

#### 0. Proposal
New York is one of the most famous places for tourists but it is also famous as an overcrowded city.
I want to figure out which kind of ads in taxi<br>
will be more effective between ads for tourists and ads for locals.

#### 1. Hypothesis
I assumed that even New York is famous, <br>
the usage rate of locals would be much higher than the usage rate of tourists in New York.<br>
Because it is city, there are many busy people.<br>
And I don't think many people ride their car in the city of well known for overcrowded city.<br>
<br>
The second hypothesis were that,<br>
In general, Tourists spend time for attractions.<br>
Locals spend time for restaurants.<br>
You don't go famous attractions if you are local, at least for me.<br>
And for tourists, it depends of which type of trip,<br> but tourists don't go restaurants like fancy restaurants,
It take so much time. Again, for me at least.<br>

#### Variables about parsing

In [ ]:
# To parse, turn the flag to True, and put directory of the trip data
# need to put slash at the end of dir
parse_ny_trip_data = False
dir_trip_data = './trip_data/'

# To export processed data, turn the flag to True, and set export directory
# need to put slash at the end of dir
export_processed_data = False
export_processed_dir = './exported_data/'
sample_dir_trip_data = './sample_trip_data/'

#### functions and mandatory value calucalations

In [ ]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import gmaps
import gmaps.datasets
import math
# pip install pyproj
# https://github.com/pyproj4/pyproj
import pyproj
from pyproj import Transformer

# read gmaps api key
with open('api.txt') as file:
    API_KEY = file.readline()
    file.close
gmaps.configure(api_key=API_KEY)


# https://jingwen-z.github.io/how-to-convert-projected-coordinates-to-latitude-longitude-by-python/
# https://stackoverflow.com/a/2689261
# https://gis.stackexchange.com/a/48952
latlong_to_world_transformer = Transformer.from_crs('epsg:4326', 'epsg:3857')
world_to_latlong_transformer = Transformer.from_crs('epsg:3857', 'epsg:4326')
# convert latitude longitude to tilemap pos
def latlong_to_world(lat_, long_):
    world_x, world_y = latlong_to_world_transformer.transform(lat_, long_)
    return world_x, world_y

# convert tilemap pos to latitude longitude
def world_to_latlong(world_x_, world_y_):
    return world_to_latlong_transformer.transform(world_x_, world_y_)

# calculate grid indices within specified boundary
def calc_grid_idx(world_x_, world_y_,
                  world_min_x_, world_min_y_,
                  world_max_x_, world_max_y_,
                  cell_step_world_,
                  grid_division_):
    grid_x = -1
    grid_y = -1
    
    if world_x_ < world_min_x_:
        return grid_x, grid_y
    elif world_x_ > world_max_x_:
        return grid_x, grid_y
    
    if world_y_ < world_min_y_:
        return grid_x, grid_y
    elif world_y_ > world_max_y_:
        return grid_x, grid_y
    
    grid_x = int((world_x_ - world_min_x_) / cell_step_world_)
    grid_y = int((world_max_y_ - world_y_) / cell_step_world_)
    
    return grid_x, grid_y

# calculate cell center position in tilemap coordinates
def get_grid_center_xy(idx_x_, idx_y_,
                       step_world_,
                       world_min_x_, world_min_y_,
                       world_max_x_, world_max_y_,
                       grid_division_):
    center_world_x = world_min_x_ + (idx_x_) * step_world_ + (step_world_ * 0.5)
    center_world_y = world_max_y_ - (idx_y_) * step_world_ - (step_world_ * 0.5)
    return center_world_x, center_world_y

# calculate weight heat grid and returns
def get_heat_grid(ref_table_,
                  lat_key_,
                  long_key_,
                  min_x_, min_y_,
                  max_x_, max_y_,
                  grid_cell_step_,
                  grid_division_):
    ret_heat_grid = pd.DataFrame(np.zeros(grid_division_ * grid_division_,
                                          dtype=int)
                                 .reshape(grid_division_, grid_division_))
    for idx, row in ref_table_.iterrows():
        hit_pos_x, hit_pos_y = latlong_to_world(row[lat_key_],
                                                row[long_key_])
        # ignore nan data
        if math.isnan(hit_pos_x) == True:
            continue
        if math.isnan(hit_pos_y) == True:
            continue    
        grid_x, grid_y = calc_grid_idx(hit_pos_x, hit_pos_y,
                                       min_x_, min_y_,
                                       max_x_, max_y_,
                                       grid_cell_step_,
                                       grid_division_)
        # ignore out of boundary
        if grid_x == -1:
            continue
        elif grid_y == -1:
            continue
        ret_heat_grid[grid_x][grid_y] += 1
    return ret_heat_grid

# get gmaps layer of points of grid cells
def gen_empty_heat_grid_layer(count_grid_,
                                world_min_x_, world_min_y_,
                                world_max_x_, world_max_y_,
                                cell_step_world_):
    heatcount_locs = []
    heatcount_weights = []
    
    for row_idx, row in count_grid_.iterrows():
        col_idx = 0
        for heat_count in row:
            if heat_count == 0:
                center_x, center_y = get_grid_center_xy(col_idx, row_idx,
                                                        cell_step_world_,
                                                        world_min_x_, world_min_y_,
                                                        world_max_x_, world_max_y_,
                                                        grid_division)
                heat_lat, heat_long = world_to_latlong(center_x, center_y)
                location = [heat_lat, heat_long]
                heatcount_locs.append(location)
                heatcount_weights.append(heat_count)
            col_idx += 1
    return gmaps.symbol_layer(heatcount_locs,
                              fill_color='black',
                               stroke_color='black',
                               scale=1)

# get gmaps layer of weight heat map
def gen_heatcount_layer(count_grid_,
                        world_min_x_, world_min_y_,
                        world_max_x_, world_max_y_,
                        cell_step_world_,
                        fill_color_,
                        stroke_color_,
                        use_heat_map_):
    heatcount_locs = []
    heatcount_weights = []
    
    for row_idx, row in count_grid_.iterrows():
        col_idx = 0
        for heat_count in row:
            if heat_count > 0:
                center_x, center_y = get_grid_center_xy(col_idx, row_idx,
                                                        cell_step_world_,
                                                        world_min_x_, world_min_y_,
                                                        world_max_x_, world_max_y_,
                                                        grid_division)
                heat_lat, heat_long = world_to_latlong(center_x, center_y)
                location = [heat_lat, heat_long]
                heatcount_locs.append(location)
                heatcount_weights.append(heat_count)
            col_idx += 1
    if use_heat_map_ == True:
        return gmaps.heatmap_layer(heatcount_locs,
                                   weights=heatcount_weights)
    else:
        return gmaps.symbol_layer(heatcount_locs,
                                  fill_color=fill_color_,
                                   stroke_color=stroke_color_,
                                   scale=1)

# get gmaps layer of boundary lines
def gen_boundary_layer(min_x_, min_y_,
                       max_x_, max_y_,
                       stroke_weight_,
                       stroke_color_):    
    boundary_lines = [[[min_x_, min_y_], [min_x_, max_y_]],
                        [[min_x_, max_y_], [max_x_, max_y_]],
                        [[max_x_, max_y_], [max_x_, min_y_]],
                        [[max_x_, min_y_], [min_x_, min_y_]]]           
    
    return gmaps.drawing_layer(features=[gmaps.Line(world_to_latlong(min_x_, min_y_),
                                                    world_to_latlong(min_x_, max_y_),
                                                    stroke_weight=stroke_weight_,
                                                    stroke_color='red'),
                                         gmaps.Line(world_to_latlong(min_x_, max_y_),
                                                    world_to_latlong(max_x_, max_y_),
                                                    stroke_weight=stroke_weight_,
                                                    stroke_color='green'),
                                         gmaps.Line(world_to_latlong(max_x_, max_y_),
                                                    world_to_latlong(max_x_, min_y_),
                                                    stroke_weight=stroke_weight_,
                                                    stroke_color='blue'),
                                         gmaps.Line(world_to_latlong(max_x_, min_y_),
                                                    world_to_latlong(min_x_, min_y_),
                                                    stroke_weight=stroke_weight_,
                                                    stroke_color='white')])

# picked ny(Manhattan) boundaries from google maps(eyeballing)
ny_zoom_level = 12
# 40.74839464368327, -73.98569953825073 Empire State Building
ny_center_lat, ny_center_long = [40.74839464368327, -73.98569953825073]

grid_division = 50

ny_center_x, ny_center_y = latlong_to_world(ny_center_lat, ny_center_long)
grid_half_size = 7000
ny_min_x = ny_center_x - grid_half_size
ny_min_y = ny_center_y - grid_half_size
ny_max_x = ny_center_x + grid_half_size
ny_max_y = ny_center_y + grid_half_size

grid_size = grid_half_size * 2
grid_cell_step = float(grid_size) / grid_division

columns_trip_data = ['dropoff_longitude', 'dropoff_latitude']

# sampled data of trip_data_1
t_sample_dropoff_1 = pd.read_csv(sample_dir_trip_data + 'trip_data_1.csv',
                                        encoding="utf-8",
                                        usecols=columns_trip_data)

dropoff_heatgrid_sample_1 = pd.read_csv(sample_dir_trip_data 
                                        + 'dropoff_heat_grid_'
                                        + str(1) + '.csv',
                                        encoding="utf-8")
# remove unused column
# https://stackoverflow.com/a/20301769
dropoff_heatgrid_sample_1 = dropoff_heatgrid_sample_1.drop(dropoff_heatgrid_sample_1.columns[0], axis=1)



#### 2. Gen data for restaurants and attractions

To get restaurants data, I make them.<br>
By searching restaurants in New York on google maps.<br>
I chose restaurants which has more highest number of ratings<br>
and type of restaurant would look like needs to spend at least a hour.<br>
And I tried to find the coordinates in front of the restaurants.<br>

In [ ]:
# best restaurants in NY table
t_ny_restaurant = pd.read_csv('./ny_restaurants.csv',
                            encoding="utf-16")
ny_restaurant_latlongs = t_ny_restaurant[['latitude', 'longitude']]
t_ny_restaurant.head()

For attractions, I used tripadvisor.com to get best list of attractions in New york.<br>
Locations of restaurant data are in Manhattan, generally,<br>
so I limit the locations of the attractions in Manhattan.<br>
And if the attraction seems to be huge and overcrowded,<br>
I grab the closest possible location for the car to drop off the passenger
by using google maps get direction feature.

In [ ]:
# best attractions in NY table
t_ny_travel = pd.read_csv('./ny_travels.csv',
                            encoding="utf-16")
ny_travel_latlongs = t_ny_travel[['latitude', 'longitude']]
t_ny_travel.head()

In [ ]:
parse_num = 6 # 6 of 12
# merge the saved dataframes
dropoff_heatgrids = []
for i in range(1, parse_num + 1):
    dropoff_heatgrid_idx = pd.read_csv(export_processed_dir 
                                        + 'dropoff_heat_grid_'
                                        + str(i) + '.csv',
                                        encoding="utf-8")
    # drop first column(generated when save)
    # https://stackoverflow.com/a/20301769
    dropoff_heatgrid_idx = dropoff_heatgrid_idx.drop(dropoff_heatgrid_idx.columns[0], axis=1)
    dropoff_heatgrids.append(dropoff_heatgrid_idx)
    

# merge heat grids by adding
merged_dropoff_heat_grid = dropoff_heatgrids[0]
for i in range(1, parse_num):
    merged_dropoff_heat_grid = merged_dropoff_heat_grid.add(dropoff_heatgrids[i])
merged_dropoff_heat_grid.head()

In [ ]:
# gmap layers

boundary_layer = gen_boundary_layer(ny_min_x, ny_min_y,
                                    ny_max_x, ny_max_y,
                                    3.0,
                                    'yellow')
empty_heat_grid = pd.DataFrame(np.zeros(grid_division * grid_division,
                                      dtype=int)
                             .reshape(grid_division, grid_division))
empty_heat_grid_layer = gen_empty_heat_grid_layer(empty_heat_grid,
                                              ny_min_x, ny_min_y,
                                              ny_max_x, ny_max_y,
                                              grid_cell_step)

ny_restaurant_latlong_layer = gmaps.symbol_layer(ny_restaurant_latlongs,
                                                  fill_color='red',
                                                  stroke_color='white',
                                                  scale=2)

ny_restaurant_heat_grid = get_heat_grid(t_ny_restaurant,
                                          'latitude',
                                          'longitude',
                                          ny_min_x, ny_min_y,
                                          ny_max_x, ny_max_y,
                                          grid_cell_step,
                                          grid_division)
ny_restaurant_heat_layer = gen_heatcount_layer(ny_restaurant_heat_grid,
                                              ny_min_x, ny_min_y,
                                              ny_max_x, ny_max_y,
                                              grid_cell_step,
                                              'white',
                                               'white',
                                               False)

ny_travel_latlong_layer = gmaps.symbol_layer(ny_travel_latlongs,
                                                  fill_color='green',
                                                  stroke_color='yellow',
                                                  scale=2)
ny_travel_heat_grid = get_heat_grid(t_ny_travel,
                                          'latitude',
                                          'longitude',
                                          ny_min_x, ny_min_y,
                                          ny_max_x, ny_max_y,
                                          grid_cell_step,
                                          grid_division)

ny_travel_heat_layer = gen_heatcount_layer(ny_travel_heat_grid,
                                              ny_min_x, ny_min_y,
                                              ny_max_x, ny_max_y,
                                              grid_cell_step,
                                              'yellow',
                                               'yellow',
                                               False)

merged_dropoff_heat_grid_layer = gen_heatcount_layer(merged_dropoff_heat_grid,
                                              ny_min_x, ny_min_y,
                                              ny_max_x, ny_max_y,
                                              grid_cell_step,
                                              'yellow',
                                               'yellow',
                                               True)
sample_dropoff_1_latlong_layer = gmaps.symbol_layer(t_sample_dropoff_1
                                                   [['dropoff_latitude',
                                                     'dropoff_longitude']],
                                                    fill_color='blue',
                                                    stroke_color='red',
                                                    scale=2)

sample_dropoff_1_heat_layer = gen_heatcount_layer(dropoff_heatgrid_sample_1,
                                                  ny_min_x, ny_min_y,
                                                  ny_max_x, ny_max_y,
                                                  grid_cell_step,
                                                  'red',
                                                   'red',
                                                  False)
                    

In [ ]:
#### 3. Analyze

3.1. Lat/Longitude of best restaurants and attractions in NY

In [ ]:
# location map of restaurants and attaractions
# White dots→Restaurants in NY
# Yellow dots→ Attractions in NY
fig = gmaps.figure(center=[ny_center_lat, ny_center_long],
                   zoom_level=ny_zoom_level)
fig.add_layer(ny_restaurant_latlong_layer)
fig.add_layer(ny_travel_latlong_layer)
fig

The restaurants(white dots) and attractions(yellow dots) are quite densed.

3.2. Grid map<br>
I used grid cell(https://gis.stackexchange.com/a/7560)<br>
to make a weighet heat map.<br>
It is about averaging points by put nearby locations to cells of grid.<br>
And to make a grid cell, I need to convert the lat/longitude to tilemap coordintes for calculating grid positions. Pyproj supports tranformation between them so I used the library.

In [ ]:
# showing grid
fig = gmaps.figure(center=[ny_center_lat, ny_center_long],
                   zoom_level=ny_zoom_level)
fig.add_layer(boundary_layer)
fig.add_layer(empty_heat_grid_layer)
fig

The square is a boundary of the grid.<br>
This is for calculating size of grid and also filter the locations within the boundaries.<br>

3.3. Projecting onto grid cell<br>
I projected the locations of restaurant and attractions.<br>
And also the grid position of them.<br>
Grid position is the small ones.

In [ ]:
# 
fig = gmaps.figure(center=[ny_center_lat, ny_center_long],
                   zoom_level=ny_zoom_level)
fig.add_layer(boundary_layer)
fig.add_layer(empty_heat_grid_layer)
fig.add_layer(ny_restaurant_latlong_layer)
fig.add_layer(ny_travel_latlong_layer)
fig.add_layer(ny_restaurant_heat_layer)
fig.add_layer(ny_travel_heat_layer)
fig

You can see how positions and grid cell locations near each other.<br>
The idea is to increment value of the cell of the location like a heatmap<br>
So each small dots are 1 or 2, black dots are 0.

3.4. About accuracy

In [ ]:
fig = gmaps.figure(center=[ny_center_lat, ny_center_long],
                   zoom_level=ny_zoom_level)
fig.add_layer(boundary_layer)
fig.add_layer(empty_heat_grid_layer)
fig.add_layer(ny_restaurant_latlong_layer)
fig.add_layer(ny_travel_latlong_layer)
fig.add_layer(ny_restaurant_heat_layer)
fig.add_layer(ny_travel_heat_layer)
fig.add_layer(sample_dropoff_1_latlong_layer)
fig.add_layer(sample_dropoff_1_heat_layer)
fig


There is a problem with resolution of grid cells.<br>
If the resolution goes high, then the accuracy goes up,<br>
so the differences of the location and grid location goes smaller.<br>
But it may not collide with taxi drop off because of it is too accurate.<br>
And it is also slow to processing the grid.<br>

However, if the resolution goes low, then the accuracy goes down.<br>
And it may share same grid coordinates between restaurant and attraction.<br>
So I need to find the adequate size.<br>
If the cell size is 50, the cell size is about the block.<br>
The processing time is about 30 mins for each month data of dropoff locations.<br>

3.4. Dropoff heat map<br>
I serialized each grid data to save time.<br>
And them each other to get merged data.<br>
And I drawed it as a weight heat map.<br>

In [ ]:
fig = gmaps.figure(center=[ny_center_lat, ny_center_long],
                   zoom_level=ny_zoom_level)
fig.add_layer(boundary_layer)
fig.add_layer(merged_dropoff_heat_grid_layer)
fig


You can see there are contrasts in the heat map.<br>
There is no heats in the locations the car can't stop or go.<br>
Like water, park, or in the middle of the bridge.<br>
And you can see very low heats on outside of Manhattan.<br>
And the center of manhattan heats very high degree.<br>



3.5. Hard to distinguish<br>

In [ ]:
fig = gmaps.figure(center=[ny_center_lat, ny_center_long],
                   zoom_level=ny_zoom_level)
fig.add_layer(boundary_layer)
fig.add_layer(merged_dropoff_heat_grid_layer)
fig.add_layer(ny_restaurant_latlong_layer)
fig.add_layer(ny_travel_latlong_layer)
fig.add_layer(ny_restaurant_heat_layer)
fig.add_layer(ny_travel_heat_layer)
fig


But every section of manhattan heats high,<br>
it is hardly to distinguish the differences between restaurants and attractions.<br>

3.5. Make a intesection<br>
I made a insersection to know influences in heatmap.<br>
I simple multiplied the heat grid and location grid because rest of are zero.<br>
So the number would be total dropoff count in cell * number of locations in cell.<br>

In [ ]:
# how many cells are matched
# need to convert due to Nan error
np_merged_dropoff_heat_grid = merged_dropoff_heat_grid.to_numpy()

np_ny_restaurant_heat_grid = ny_restaurant_heat_grid.to_numpy()
np_intersection_restaurant_merged = np_merged_dropoff_heat_grid * np_ny_restaurant_heat_grid
intersection_restaurant_merged = pd.DataFrame(np_intersection_restaurant_merged)

np_ny_travel_heat_grid = ny_travel_heat_grid.to_numpy()
np_intersection_travel_merged = np_merged_dropoff_heat_grid * ny_travel_heat_grid
intersection_travel_merged = pd.DataFrame(np_intersection_travel_merged)

In [ ]:
intersection_restaurant_merged_heat_layer = gen_heatcount_layer(intersection_restaurant_merged,
                                                  ny_min_x, ny_min_y,
                                                  ny_max_x, ny_max_y,
                                                  grid_cell_step,
                                                  'green',
                                                   'green',
                                                  True)
intersection_travel_merged_heat_layer = gen_heatcount_layer(intersection_travel_merged,
                                                  ny_min_x, ny_min_y,
                                                  ny_max_x, ny_max_y,
                                                  grid_cell_step,
                                                  'green',
                                                   'green',
                                                  True)

In [ ]:
fig = gmaps.figure(center=[ny_center_lat, ny_center_long],
                   zoom_level=ny_zoom_level)
fig.add_layer(boundary_layer)
fig.add_layer(intersection_restaurant_merged_heat_layer)
fig

In [ ]:
fig = gmaps.figure(center=[ny_center_lat, ny_center_long],
                   zoom_level=ny_zoom_level)
fig.add_layer(boundary_layer)
fig.add_layer(intersection_travel_merged_heat_layer)
fig

Each intersection of heat maps.

#### 4. Conclusion

In [ ]:
cell_count_merged_heat = merged_dropoff_heat_grid.astype(bool).values.sum()
intersection_percent_heat = (cell_count_merged_heat / (grid_division * grid_division)) * 100.0
print('heat cell count of dropoffs: ', intersection_percent_heat, '% of total cell count')

In [ ]:
# https://stackoverflow.com/a/34156147
cell_count_restaurants_merged = intersection_restaurant_merged.astype(bool).values.sum()
print('# of restaurant heat cell count: ', cell_count_restaurants_merged)
intersection_percent_restaurants = (cell_count_restaurants_merged / cell_count_merged_heat) * 100.0
print('heat cell count of restaurants: ', intersection_percent_restaurants, '% of dropoff cells')

In [ ]:
# https://stackoverflow.com/a/34156147
cell_count_travel_merged = intersection_travel_merged.astype(bool).values.sum()
print('# of attraction heat cell count: ', cell_count_travel_merged)
intersection_percent_travels = (cell_count_travel_merged / cell_count_merged_heat) * 100.0
print('heat cell count of attractions: ', intersection_percent_travels, '% of dropoff cells')
cell_count_travel_merged

In [ ]:
intersection_restaurant_score = intersection_restaurant_merged.values.sum()
print('restaurants total heat score: ', intersection_restaurant_score)

In [ ]:
intersection_travel_score = intersection_travel_merged.values.sum()
print('attractions total heat score: ', intersection_travel_score)

In [ ]:
if intersection_restaurant_score > intersection_travel_score:
    print('restaurants total heat score is higher than attractions')
elif intersection_restaurant_score < intersection_travel_score:
    print('attractions total heat score is higher than restaurants')
else:
    print('attractions total heat score and restaurants total heat score have same score')

In [ ]:
print('Restaurants and attractions have same heat cell count,')
print('but attractions have more score than restaurants by: ', intersection_travel_score - intersection_restaurant_score)
print('Therefore, tourists seems use cabs more than locals in NY')

#### Parsing(optional, huge cost of calculations)

In [ ]:
if parse_ny_trip_data == True:
    # parse table
    t_dropoff_1 = pd.read_csv(dir_trip_data 
                                + 'trip_data_'
                                + str(1) + '.csv',
                                encoding="utf-8",
                                usecols=columns_trip_data)


In [ ]:
 if export_processed_data == True:
    # gen heat grid of parsed table(lots of calculations)
    t_dropoff_heat_grid_1 = get_heat_grid(t_dropoff_1,
                                              'dropoff_latitude',
                                              'dropoff_longitude',
                                              ny_min_x, ny_min_y,
                                              ny_max_x, ny_max_y,
                                              grid_cell_step,
                                              grid_division)
    # export to csv
    t_dropoff_heat_grid_1.to_csv(export_processed_dir
                                   + 'dropoff_heat_grid_'
                                   + str(1) + '.csv')

In [ ]:
if parse_ny_trip_data == True:
    t_dropoff_2 = pd.read_csv(dir_trip_data 
                                + 'trip_data_'
                                + str(2) + '.csv',
                                encoding="utf-8",
                                usecols=columns_trip_data)

In [ ]:
 if export_processed_data == True:
    t_dropoff_heat_grid_2 = get_heat_grid(t_dropoff_2,
                                              'dropoff_latitude',
                                              'dropoff_longitude',
                                              ny_min_x, ny_min_y,
                                              ny_max_x, ny_max_y,
                                              grid_cell_step,
                                              grid_division)
    t_dropoff_heat_grid_2.to_csv(export_processed_dir
                               + 'dropoff_heat_grid_'
                               + str(2) + '.csv')

In [ ]:
if parse_ny_trip_data == True:
    columns = [' dropoff_longitude', ' dropoff_latitude']
    t_dropoff_3 = pd.read_csv(dir_trip_data 
                                + 'trip_data_'
                                + str(3) + '.csv',
                                encoding="utf-8",
                                usecols=columns)

In [ ]:
 if export_processed_data == True:
    t_dropoff_heat_grid_3 = get_heat_grid(t_dropoff_3,
                                              ' dropoff_latitude',
                                              ' dropoff_longitude',
                                              ny_min_x, ny_min_y,
                                              ny_max_x, ny_max_y,
                                              grid_cell_step,
                                              grid_division)
    t_dropoff_heat_grid_3.to_csv(export_processed_dir
                                 + 'dropoff_heat_grid_'
                                 + str(3) + '.csv')

In [ ]:
if parse_ny_trip_data == True:
    columns = [' dropoff_longitude', ' dropoff_latitude']
    t_dropoff_4 = pd.read_csv(dir_trip_data 
                                + 'trip_data_'
                                + str(4) + '.csv',
                                encoding="utf-8",
                                usecols=columns)

In [ ]:
 if export_processed_data == True:
    t_dropoff_heat_grid_4 = get_heat_grid(t_dropoff_4,
                                              ' dropoff_latitude',
                                              ' dropoff_longitude',
                                              ny_min_x, ny_min_y,
                                              ny_max_x, ny_max_y,
                                              grid_cell_step,
                                              grid_division)
    t_dropoff_heat_grid_4.to_csv(export_processed_dir
                                 + 'dropoff_heat_grid_'
                                 + str(4) + '.csv')

In [ ]:
if parse_ny_trip_data == True:
    columns = [' dropoff_longitude', ' dropoff_latitude']
    t_dropoff_5 = pd.read_csv(dir_trip_data 
                                + 'trip_data_'
                                + str(5) + '.csv',
                                encoding="utf-8",
                                usecols=columns)

In [ ]:
 if export_processed_data == True:
    t_dropoff_heat_grid_5 = get_heat_grid(t_dropoff_5,
                                              ' dropoff_latitude',
                                              ' dropoff_longitude',
                                              ny_min_x, ny_min_y,
                                              ny_max_x, ny_max_y,
                                              grid_cell_step,
                                              grid_division)
    t_dropoff_heat_grid_5.to_csv(export_processed_dir
                                 + 'dropoff_heat_grid_'
                                 + str(5) + '.csv')

In [ ]:
if parse_ny_trip_data == True:
    columns = [' dropoff_longitude', ' dropoff_latitude']
    t_dropoff_6 = pd.read_csv(dir_trip_data 
                                + 'trip_data_'
                                + str(6) + '.csv',
                                encoding="utf-8",
                                usecols=columns)

In [ ]:
 if export_processed_data == True:
    t_dropoff_heat_grid_6 = get_heat_grid(t_dropoff_6,
                                              ' dropoff_latitude',
                                              ' dropoff_longitude',
                                              ny_min_x, ny_min_y,
                                              ny_max_x, ny_max_y,
                                              grid_cell_step,
                                              grid_division)
    t_dropoff_heat_grid_6.to_csv(export_processed_dir
                                 + 'dropoff_heat_grid_'
                                 + str(6) + '.csv')